仮想環境にはいる！
source .venv/bin/activate

Windows
Set-ExecutionPolicy RemoteSigned -Scope CurrentUser
.\.venv\Scripts\activate

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルのロード
# modelName = "cyberagent/open-calm-3b"
modelName = "cyberagent/open-calm-small"

model = AutoModelForCausalLM.from_pretrained("tunedModels/LoRA/TEST/checkpoint-300")
tokenizer = AutoTokenizer.from_pretrained(modelName)

#CUDAかMPSの確認
#コンピュータにとって最適な環境を自動的に設定する
if torch.cuda.is_available():
    print("CUDA is available")
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
    
elif torch.backends.mps.is_available():
    print("MPS is available")
    device = torch.device("mps")
    device_name = "Apple Silicon (MPS)"
    
else:
    print("No CUDA or MPS support available")
    device = torch.device("cpu")
    device_name = "CPU"
    
print(f"Using device: {device_name}")

MPS is available
Using device: Apple Silicon (MPS)


In [33]:
# プロンプトからテキストを生成する
prompt = "質問をしてください"

input = tokenizer(prompt, return_tensors="pt")

tokens = model.generate(
    **input,
    max_new_tokens=50,
    do_sample=False,
).to(device)

output = tokenizer.decode(tokens[0], skip_special_tokens=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


質問をしてください。質問に答える形で、質問をしていきます。
Q.  質問は、どのような内容ですか?A.                           
